## Scraping banten.antarnews.com

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
from urllib.request import Request, urlopen

In [2]:
jumlah_index = 50
threads_link = []
links = []
results = []
threads = []

In [3]:
def scrape_links(page_number):
    url = f"https://banten.antaranews.com/seputar-banten/{page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article',{"class": "simple-post simple-big clearfix"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [4]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 10 links from page 8
Scraped 10 links from page 12
Scraped 10 links from page 4
Scraped 10 links from page 7
Scraped 10 links from page 22
Scraped 10 links from page 6
Scraped 10 links from page 10
Scraped 10 links from page 14
Scraped 10 links from page 2
Scraped 10 links from page 34
Scraped 10 links from page 43
Scraped 10 links from page 28
Scraped 9 links from page 1
Scraped 10 links from page 30
Scraped 10 links from page 19
Scraped 10 links from page 46
Scraped 10 links from page 38
Scraped 10 links from page 48
Scraped 10 links from page 18
Scraped 10 links from page 45
Scraped 10 links from page 24
Scraped 10 links from page 11
Scraped 10 links from page 37
Scraped 10 links from page 29
Scraped 10 links from page 49
Scraped 10 links from page 20
Scraped 10 links from page 3
Scraped 10 links from page 5
Scraped 10 links from page 42
Scraped 10 links from page 36
Scraped 10 links from page 32
Scraped 10 links from page 13
Scraped 10 links from page 16
Scraped 10 links fr

In [5]:
def scrape_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "post-title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "tags-wrapper"})
            if author_elem:
                author_text = author_elem.find('span')
                author_text = author_text.text.split(':')[1]
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('span', {"class": "article-date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            # category_elements = soup.find('ul', {"class": "breadcrumb"})
            # if category_elements:
            #     category_text = category_elements.find_all('li')
            #     category_text= category_text[2].get_text()
            # else:
            category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "post-content clearfix"})
            
            if body_elem:
                content_elem = body_elem.find_all('div')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'region':'banten',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [6]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

Error fetching URL 'https://banten.antaranews.com/berita/250290/jadwal-sim-keliling-di-banten-lokasi-dan-biaya': HTTPSConnectionPool(host='banten.antaranews.com', port=443): Max retries exceeded with url: /berita/250290/jadwal-sim-keliling-di-banten-lokasi-dan-biaya (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f51a06d8e90>: Failed to establish a new connection: [Errno 111] Connection refused'))
Error fetching URL 'https://banten.antaranews.com/berita/256230/warga-diingatkan-tak-bakar-sampah-sembarangan-karena-bisa-picu-kebakaran': HTTPSConnectionPool(host='banten.antaranews.com', port=443): Max retries exceeded with url: /berita/256230/warga-diingatkan-tak-bakar-sampah-sembarangan-karena-bisa-picu-kebakaran (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f51c2f3fb50>: Failed to establish a new connection: [Errno 111] Connection refused'))
Error fetching URL 'https://banten.antaranews.com/berita/251382/penjual-bendera-as

In [7]:
df = pd.DataFrame(results)
print(len(results))
df.head(100)

415


,title,author,category,date,content,region,link
0,DPRD setujui Raperda Perubahan APBD Kabupaten ...,Desi Purnama Sari,Category not found,"Jumat, 15 September 2023 5:36 WIB",\nBerdasarkan keterangan resmi yang diterima p...,banten,https://banten.antaranews.com/berita/256962/dp...
1,Tim SAR Banten sisir warga yang diduga tenggel...,Mansyur suryana,Category not found,"Rabu, 13 September 2023 9:00 WIB",Tim Pencarian dan Pertolongan (SAR) Banten mel...,banten,https://banten.antaranews.com/berita/256707/ti...
2,Nama 45 calon yang lolos tes akademik Komite I...,.,Category not found,"Sabtu, 23 September 2023 11:35 WIB",Content not found,banten,https://banten.antaranews.com/berita/258090/na...
3,Kadindik Banten tindaklanjuti arahan Pj Gubern...,Mulyana,Category not found,"Kamis, 20 Juli 2023 17:51 WIB",Content not found,banten,https://banten.antaranews.com/berita/249819/ka...
4,Tim gabungan masih berupaya padamkan api kapal...,Desi Purnama Sari,Category not found,"Kamis, 7 September 2023 13:45 WIB",Kepulan asap hitam pekat masih membumbung pada...,banten,https://banten.antaranews.com/berita/255978/ti...
...,...,...,...,...,...,...,...
95,Polres Lebak jemput bola edukasi tertib berlal...,Mansyur suryana,Category not found,"Kamis, 21 September 2023 8:11 WIB",Kepolisian Resor (Polres) Lebak Polda Banten m...,banten,https://banten.antaranews.com/berita/257751/po...
96,Aplikasi Pujangga pengadilan agama sekarang te...,Achmad Irfan,Category not found,"Jumat, 4 Agustus 2023 10:42 WIB",Content not found,banten,https://banten.antaranews.com/berita/251736/ap...
97,50 petugas disiagakan jaga kebersihan kawasan ...,Achmad Irfan,Category not found,"Jumat, 21 Juli 2023 13:32 WIB",Content not found,banten,https://banten.antaranews.com/berita/249924/50...
98,BMKG imbau pelaku pelayaran waspadai gelombang...,Mansyur suryana,Category not found,"Selasa, 15 Agustus 2023 10:14 WIB","Badan Meteorologi, Klimatologi, dan Geofisika ...",banten,https://banten.antaranews.com/berita/252933/bm...


Tempat Simpan Save

In [ ]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'./excel/banten_antranews_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')